In [1]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

# from sklearn.linear_model import LinearRegression ## likes, dislikes 대체
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

import os
from dotenv import load_dotenv

c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
file_path = 'C:/py_src/awake/data/'

In [3]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv(file_path + 'merge_df_users_final3.csv', low_memory=False)
# youtube_videos = pd.read_csv(file_path + 'youtube_videos_final2.csv')

## 계정 데이터 분석

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [7]:
# 컬럼 정리
unique_col = merge_df_users_fin.columns[:11]
x_col = merge_df_users_fin.columns[11:].drop('estimated_ad_revenue') ## y값 제거

In [8]:
# 데이터 분할
train_data = merge_df_users_fin[merge_df_users_fin['date'] <= '2024-02-11']
test_data = merge_df_users_fin[merge_df_users_fin['date'] > '2024-02-11']

In [9]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(84473, 57)
(21210, 57)


### 모델 기법 적용

In [10]:
# 각 모델 정의
rf_model = RandomForestRegressor(random_state=42)
gbm_model = GradientBoostingRegressor(random_state=42)
lgbm_model = LGBMRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)

# 모델 리스트
models = {
    'RandomForest': rf_model,
    'GBM': gbm_model,
    'LightGBM': lgbm_model,
    'XGBoost': xgb_model
}

In [9]:
# # 모델별 교차 검증 결과 저장
# results = {}

# for model_name, model in models.items():
#     print(f"\n{model_name} 모델 성능 평가 중...")
    
#     # 교차 검증
#     cv_scores = cross_val_score(model, train_data[x_col], train_data['subscribers_count'], cv=5, scoring='neg_mean_squared_error')
    
#     # 평균 RMSE 계산
#     rmse_scores = np.sqrt(-cv_scores)  # neg_mean_squared_error는 음수이므로 양수로 변환 후 제곱근
#     mean_rmse = rmse_scores.mean()
#     print(f"{model_name} 교차 검증 평균 RMSE: {mean_rmse}")
    
#     # 결과 저장
#     results[model_name] = mean_rmse

### 모델 성능 평가

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

for model in models:
    # 모델 정의 및 학습
    models[model].fit(train_data[x_col], train_data['estimated_ad_revenue'])

    # 예측
    y_pred = models[model].predict(test_data[x_col])

    # MSE 계산
    mse = mean_squared_error(test_data['estimated_ad_revenue'], y_pred)

    # RMSE 계산
    rmse = np.sqrt(mse)

    # R² 값 계산
    r2 = r2_score(test_data['estimated_ad_revenue'], y_pred)

    # 결과 출력
    print(model)
    print(f"R² 값: {r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print("----------------------------------------")
    print("")

RandomForest
R² 값: 0.9925
MSE: 54900603.8218
RMSE: 7409.4942
----------------------------------------

GBM
R² 값: 0.9948
MSE: 38151142.2963
RMSE: 6176.6611
----------------------------------------

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048380 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11454
[LightGBM] [Info] Number of data points in the train set: 84473, number of used features: 45
[LightGBM] [Info] Start training from score 14667.681766
LightGBM
R² 값: 0.9225
MSE: 569941669.8060
RMSE: 23873.4511
----------------------------------------

XGBoost
R² 값: 0.9652
MSE: 256061983.5355
RMSE: 16001.9369
----------------------------------------



In [92]:
# # 비선형모델 활용 변수 선정
# importances_df = pd.DataFrame({
#     'features': x_col,
#     'rf_importance': models['RandomForest'].feature_importances_,
#     'gbm_importance': models['GBM'].feature_importances_,
#     'lgbm_importance': models['LightGBM'].feature_importances_,
#     'xgb_importance': models['XGBoost'].feature_importances_    
# })
# importances_df['lgbm_importance'] = importances_df['lgbm_importance'] / importances_df['lgbm_importance'].sum() ## 다른 모델과 중요도 단위 다르기 때문에 정규화
# importances_df['mean_importance'] = importances_df[['rf_importance', 'gbm_importance', 'lgbm_importance', 'xgb_importance']].mean(axis=1) ## 각 변수별 모델 중요도 평균

### 모델 성능 개선

스태킹

In [14]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# 스태킹 모델 정의
stacking_model = StackingRegressor(
    estimators=[
    ('random_forest', models['RandomForest']),
    ('gbm', models['GBM']),
    ('lightgbm', models['LightGBM']),
    ('xgboost', models['XGBoost'])
    ],
    final_estimator=Ridge()
)

# 스태킹 모델 학습
stacking_model.fit(train_data[x_col], train_data['estimated_ad_revenue'])

# 예측
y_pred = stacking_model.predict(test_data[x_col])

# 성능 평가
# R² 값 계산
r2 = r2_score(test_data['estimated_ad_revenue'], y_pred)

# MSE 계산
mse = mean_squared_error(test_data['estimated_ad_revenue'], y_pred)

# RMSE 계산
rmse = mean_squared_error(test_data['estimated_ad_revenue'], y_pred, squared=False)

print(f"스태킹 앙상블 모델 R2: {r2:.4f}")
print(f"스태킹 앙상블 모델 MSE: {mse:.4f}")
print(f"스태킹 앙상블 모델 RMSE: {rmse:.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11454
[LightGBM] [Info] Number of data points in the train set: 84473, number of used features: 45
[LightGBM] [Info] Start training from score 14667.681766
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11453
[LightGBM] [Info] Number of data points in the train set: 67578, number of used features: 45
[LightGBM] [Info] Start training from score 17638.612776
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11455
[LightGBM] [Info] Number of data points in the train set: 67578, number of used features: 45
[LightGBM] [Info

c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 실제데이터 결과 확인

In [15]:
# 예측 결과 확인
y_pred = stacking_model.predict(merge_df_users_fin[x_col])
merge_df_users_fin['predict'] = y_pred

In [16]:
# 계정별 구독자수 평균, 구독자 예측수 평균 비교
result_df = merge_df_users_fin[['youtube_user_id','date','channel_title','estimated_ad_revenue','predict']]

# 1개월, 3개월, 6개월, 1년 뒤 구독자수 예측값
result_df['date'] = pd.to_datetime(result_df['date']) 

result_df['1_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-30).rolling(window=30).mean()
result_df['3_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-90).rolling(window=90).mean()
result_df['6_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-180).rolling(window=180).mean()
result_df['12_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-365).rolling(window=365).mean()

# 계정별 기간에 따른 구독자수 예측값 비교
result_df_final = result_df.groupby(['youtube_user_id'])[['estimated_ad_revenue','predict','1_month_future_predict','3_month_future_predict','6_month_future_predict','12_month_future_predict']].mean().reset_index()

C:\Users\kwan\AppData\Local\Temp\ipykernel_4396\2106748872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['date'] = pd.to_datetime(result_df['date'])
C:\Users\kwan\AppData\Local\Temp\ipykernel_4396\2106748872.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['1_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-30).rolling(window=30).mean()
C:\Users\kwan\AppData\Local\Temp\ipykernel_4396\2106748872.py:8: SettingWithCopyWarning: 
A value is trying to be

In [17]:
# 기간 따른 예측 결과
# 1개월
result_1_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] < result_df_final[result_df_final['predict'] > 0]['1_month_future_predict']]['youtube_user_id']
result_1_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] > result_df_final[result_df_final['predict'] > 0]['1_month_future_predict']]['youtube_user_id']

# 3개월
result_3_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] < result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']]['youtube_user_id']
result_3_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] > result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']]['youtube_user_id']

# 6개월
result_6_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] < result_df_final[result_df_final['predict'] > 0]['6_month_future_predict']]['youtube_user_id']
result_6_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] > result_df_final[result_df_final['predict'] > 0]['6_month_future_predict']]['youtube_user_id']

# 12개월
result_12_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] < result_df_final[result_df_final['predict'] > 0]['12_month_future_predict']]['youtube_user_id']
result_12_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['estimated_ad_revenue'] > result_df_final[result_df_final['predict'] > 0]['12_month_future_predict']]['youtube_user_id']

In [31]:
print(len(result_df[result_df['youtube_user_id'].isin(result_6_month_minus_user_id)]['channel_title'].unique()))
print(result_df[result_df['youtube_user_id'].isin(result_6_month_minus_user_id)]['channel_title'].unique())

112
['성팩 SPAAK' '0' '띠혜 ddihye' '소리미의 신화방송' 'Ella' '찌늉' 'hyeppening 혜프닝'
 'KIMBEE 킴비' '나는 불독' '지니원장의피부톡톡' '오늘도희다 HEEDA' '낭만아저씨코디TV'
 'Kevin Factory케빈팩토리' 'assesta' 'JinBlog 진블로그' '흙회장' '차세이CHASEYi'
 'Mind Patting마음토닥' '배우GO' '시골낭만아재' 'Seol-A 라이더 설아' 'SATUR 세터업'
 '굥플레이스 맛집투어' '이숲soop' '래띠 LAETI' '시리얼 Sireal' '채찍단' '원의 독백'
 '잉툰TV- 만화로 쉽게 영어배우자' '라이프에이치 Life.H' '채림처럼firstcherry'
 '오엔티엘 패션 / ONTL FASHION' '에디레일 Eddy Rails' '모하지연 MOHAJIYEON'
 '시현하다 RECORDERS' '데일리 슬슬' '오토컨테이너 스튜디오' '부반TV_부에 반하다' '미디하는남자'
 '바라던 바다 BADACHANNEL' '여정을떠난여정' '찌수연' '월텍남 - 월스트리트 테크남' '팀브라더스' '이고 EGO'
 'kiu기우쌤' '콤므' '수빙수tv sooBingsoo' '카이바군' '-mentalholder 멘탈홀더 tv'
 'Suzevi ASMR' '사부작장제소sabujakfarrier' '지미 geemi.' '아무튼 리뷰어즈' '만능혁키'
 '꽃 읽어주는 남자 kkotnam' '코드맨TV' '두남자 토익TV' '너드 슬로리 SloLee'
 '석시원 커플 SeokSiWon Couple' '태권민국_Captain Master' '임영곤 게임방송'
 '키메키친 Kime_kitchen' '너굴몬' '디지털생활제안' '주대성 JooDaesung' '나는미도'
 '용싸부 yongssaboo' '미니멀영어 Minimal English' '채채ChaeChae' '뚜니랑' '캠핑 릴리아빠'
 '비단한생활 Bidan' '나나무비' '윤순의 평범치 않은 생활' '청어

In [35]:
# 계속 증가
increase_user_id = result_df_final[result_df_final['predict'] > 0][(result_df_final[result_df_final['predict'] > 0]['1_month_future_predict'] < result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']) & 
                                                                   (result_df_final[result_df_final['predict'] > 0]['3_month_future_predict'] < result_df_final[result_df_final['predict'] > 0]['6_month_future_predict'])]['youtube_user_id']
print(len(result_df[result_df['youtube_user_id'].isin(increase_user_id)]['channel_title'].unique()))
print(result_df[result_df['youtube_user_id'].isin(increase_user_id)]['channel_title'].unique())

51
['Ella' '0' '차박씬' 'assesta' '오디디 코미디' '황나겸' '래아TV' '시리얼 Sireal'
 'abbapraise 아바프레이즈' '원의 독백' '오토컨테이너 스튜디오' '라나제이베이킹Lana J' '군대위키' '미디하는남자'
 'Jeffreyxking' '유경몬' '콜드쉽 Coldsheep' '라이라마' '-mentalholder 멘탈홀더 tv' '도도'
 '퓨츠앙' '엔트리뷰 [누구나 재미있는 테크리뷰]' 'Dalhae달달해' '단곰' '앙찡' '코인덕 차트아지'
 '비단한생활 Bidan' '황헬린 탈출기' '보미름' '日本ジヌ【니혼지누】ー韓国に関する全て'
 "파파스캠핑 papa's camp | a korean camper" '정케빈 KEVIN' '요니의 응원 yoni' 'MORE김모어'
 '니들needle' '유네린NERIN' '복지다있소' '밖비타' '후니트립 hoony_trip'
 '와뷰티TV | Wow Beauty ASMR' '김퍼프PUFF' '쿜쿜쿜' '주당 김자케' '이지호이죠_JiHo' '에피코딩'
 '키나kkina' '동아일보' '이현우의 MLBTV' '평범한 사업가' '여리여리YeoriYeori' 'JN테크리뷰']


In [36]:
# 계속 감소
decrease_user_id = result_df_final[result_df_final['predict'] > 0][(result_df_final[result_df_final['predict'] > 0]['1_month_future_predict'] > result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']) & 
                                                                   (result_df_final[result_df_final['predict'] > 0]['3_month_future_predict'] > result_df_final[result_df_final['predict'] > 0]['6_month_future_predict'])]['youtube_user_id']
print(len(result_df[result_df['youtube_user_id'].isin(decrease_user_id)]['channel_title'].unique()))
print(result_df[result_df['youtube_user_id'].isin(decrease_user_id)]['channel_title'].unique())

60
['소리미의 신화방송' '0' '찌늉' 'hyeppening 혜프닝' '오늘도희다 HEEDA' 'JinBlog 진블로그' '흙회장'
 'Mind Patting마음토닥' '시골낭만아재' 'Seol-A 라이더 설아' 'SATUR 세터업' '굥플레이스 맛집투어'
 '채찍단' '잉툰TV- 만화로 쉽게 영어배우자' '라이프에이치 Life.H' '오엔티엘 패션 / ONTL FASHION'
 '모하지연 MOHAJIYEON' '시현하다 RECORDERS' '부반TV_부에 반하다' '바라던 바다 BADACHANNEL'
 '여정을떠난여정' '찌수연' '팀브라더스' '이고 EGO' 'kiu기우쌤' '맛집남자 foodman' '카이바군'
 'Suzevi ASMR' '사부작장제소sabujakfarrier' '지미 geemi.' '아무튼 리뷰어즈' '임영곤 게임방송'
 '키메키친 Kime_kitchen' '너굴몬' '주대성 JooDaesung' '나는미도' '미니멀영어 Minimal English'
 '채채ChaeChae' '캠핑 릴리아빠' '윤순의 평범치 않은 생활' '청어람ARMC' 'Ood 오드' '전또'
 '소피요가 Sophie Yoga' '벽돌할아버지 Brick grandpa' '믿식당' '집구석구석꿀팁, 집꿀' '꿈꾼 배기'
 '나연이즈백 LPGA Na Yeon Choi' '세계여행 테리로그 TERRYLOG' '막셋의 종합게임' '스타트업잡스'
 'MerryMa 메리마' '모리캠핑' '구봉바다낚시 뽀식이' '빅민 GAME' '잼스기타' 'OSSC' 'Lizzy리지'
 '신크TV']


## 콘텐츠 데이터 분석

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [14]:
youtube_videos.columns

Index(['youtube_user_id', 'video', 'end_date', 'quarterly_avg_view_percentage',
       'playlist_removal_rate', 'ad_revenue_rate',
       'weekly_avg_view_percentage', 'playback_rate', 'dislikes', 'views',
       'comment_rate', 'quarterly_avg_view_duration',
       'weekly_total_engagement', 'playlist_addition_rate',
       'positive_engage_rate', 'share_rate', 'estimatedRedPartnerRevenue',
       'weekly_watch_time', 'monthly_total_engagement',
       'watch_time_loss_per_playlist_remove', 'monthly_watch_time',
       'quarterly_estimated_revenue', 'subscribers_lost_per_playlist_remove',
       'subscribers_gained_per_playlist_add', 'averageViewDuration',
       'monthly_playlist_change_rate', 'shares', 'weekly_playlist_change_rate',
       'weekly_engagement_rate', 'dislike_rate', 'videosRemovedFromPlaylists',
       'redViews', 'averageViewPercentage', 'weekly_videos_removed',
       'monthly_views', 'subscribers_conversion_rate',
       'estimatedMinutesWatched', 'like_rate', 'vid

In [15]:
# 컬럼 정리
unique_col = youtube_videos.columns[:3]
x_col = youtube_videos.columns[3:].drop(['net_subscribers_change']) ## y값 제거

In [16]:
# 데이터 분할
train_data = youtube_videos[youtube_videos['end_date'] <= '2024-02-11']
test_data = youtube_videos[youtube_videos['end_date'] > '2024-02-11']

In [17]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(6538190, 62)
(2031321, 62)


### 모델 기법 적용

In [18]:
# 각 모델 정의
rf_model = RandomForestRegressor(random_state=42)
gbm_model = GradientBoostingRegressor(random_state=42)
lgbm_model = LGBMRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)

# 모델 리스트
models = {
    'RandomForest': rf_model,
    'GBM': gbm_model,
    'LightGBM': lgbm_model,
    'XGBoost': xgb_model
}

In [ ]:
# # 모델별 교차 검증 결과 저장
# results = {}

# for model_name, model in models.items():
#     print(f"\n{model_name} 모델 성능 평가 중...")
    
#     # 교차 검증
#     cv_scores = cross_val_score(model, train_data[x_col], train_data['subscribers_count'], cv=5, scoring='neg_mean_squared_error')
    
#     # 평균 RMSE 계산
#     rmse_scores = np.sqrt(-cv_scores)  # neg_mean_squared_error는 음수이므로 양수로 변환 후 제곱근
#     mean_rmse = rmse_scores.mean()
#     print(f"{model_name} 교차 검증 평균 RMSE: {mean_rmse}")
    
#     # 결과 저장
#     results[model_name] = mean_rmse

### 모델 성능 평가

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

for model in models:
    # 모델 정의 및 학습
    models[model].fit(train_data[x_col], train_data['net_subscribers_change'])

    # 예측
    y_pred = models[model].predict(test_data[x_col])

    # MSE 계산
    mse = mean_squared_error(test_data['net_subscribers_change'], y_pred)

    # RMSE 계산
    rmse = np.sqrt(mse)

    # R² 값 계산
    r2 = r2_score(test_data['net_subscribers_change'], y_pred)

    # 결과 출력
    print(model)
    print(f"R² 값: {r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print("----------------------------------------")
    print("")

In [ ]:
# # 비선형모델 활용 변수 선정
# importances_df = pd.DataFrame({
#     'features': x_col,
#     'rf_importance': models['RandomForest'].feature_importances_,
#     'gbm_importance': models['GBM'].feature_importances_,
#     'lgbm_importance': models['LightGBM'].feature_importances_,
#     'xgb_importance': models['XGBoost'].feature_importances_    
# })
# importances_df['lgbm_importance'] = importances_df['lgbm_importance'] / importances_df['lgbm_importance'].sum() ## 다른 모델과 중요도 단위 다르기 때문에 정규화
# importances_df['mean_importance'] = importances_df[['rf_importance', 'gbm_importance', 'lgbm_importance', 'xgb_importance']].mean(axis=1) ## 각 변수별 모델 중요도 평균

### 모델 성능 개선

In [ ]:
x_col

Index(['weekly_estimated_revenue', 'weekly_estimated_ad_revenue',
       'subscriber_increase_rate', 'monthly_revenue_per_ad_impression',
       'revenue_per_subscriber', 'quarterly_avg_view_duration',
       'quarterly_estimated_revenue', 'monthly_net_subscribers_change',
       'estimated_revenue', 'subscriber_view_time_rate',
       'quarterly_total_view_time', 'weekly_revenue_per_ad_impression',
       'monthly_subscribers_lost', 'subscribers_lost', 'watched_time_rate',
       'age18-24.female', 'quarterly_subscribers_gained', 'gross_revenue',
       'gross_revenue_per_ad_impression', 'weekly_net_subscribers_change',
       'playback_rate', 'age25-34.female', 'estimated_red_partner_revenue',
       'redViews', 'quarterly_estimated_ad_revenue',
       'quarterly_net_subscribers_change', 'estimatedMinutesWatched',
       'subscriber_decrease_rate', 'weekly_subscribers_lost',
       'estimated_ad_revenue', 'quarterly_subscribers_lost',
       'monthly_subscribers_gained', 'age13-17.fe

스태킹

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# 스태킹 모델 정의
stacking_model = StackingRegressor(
    estimators=[
    ('random_forest', models['RandomForest']),
    # ('gbm', models['GBM']),
    # ('lightgbm', models['LightGBM']),
    ('xgboost', models['XGBoost'])
    ],
    final_estimator=Ridge()
)

# 스태킹 모델 학습
stacking_model.fit(train_data[x_col], train_data['subscribers_count'])

# 예측
y_pred = stacking_model.predict(test_data[x_col])

# 성능 평가
# R² 값 계산
r2 = r2_score(test_data['subscribers_count'], y_pred)

# MSE 계산
mse = mean_squared_error(test_data['subscribers_count'], y_pred)

# RMSE 계산
rmse = mean_squared_error(test_data['subscribers_count'], y_pred, squared=False)

print(f"스태킹 앙상블 모델 R2: {r2:.4f}")
print(f"스태킹 앙상블 모델 MSE: {mse:.4f}")
print(f"스태킹 앙상블 모델 RMSE: {rmse:.4f}")

스태킹 앙상블 모델 R2: 0.9661
스태킹 앙상블 모델 MSE: 1998305072.9542
스태킹 앙상블 모델 RMSE: 44702.4057


c:\Users\kkims\anaconda3\envs\awake\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 실제데이터 결과 확인

In [ ]:
# 예측 결과 확인
y_pred = stacking_model.predict(merge_df_users_fin[x_col])
merge_df_users_fin['predict'] = y_pred

In [ ]:
# 계정별 구독자수 평균, 구독자 예측수 평균 비교
result_df = merge_df_users_fin[['youtube_user_id','date','channel_title','subscriberCount','subscribers_count','predict']]

# 1개월, 3개월, 6개월, 1년 뒤 구독자수 예측값
result_df['date'] = pd.to_datetime(result_df['date']) 

result_df['1_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-30).rolling(window=30).mean()
result_df['3_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-90).rolling(window=90).mean()
result_df['6_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-180).rolling(window=180).mean()
result_df['12_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-365).rolling(window=365).mean()

# 계정별 기간에 따른 구독자수 예측값 비교
result_df_final = result_df.groupby(['youtube_user_id'])[['subscribers_count','predict','1_month_future_predict','3_month_future_predict','6_month_future_predict','12_month_future_predict']].mean().reset_index()

C:\Users\kkims\AppData\Local\Temp\ipykernel_23628\2617090873.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['date'] = pd.to_datetime(result_df['date'])
C:\Users\kkims\AppData\Local\Temp\ipykernel_23628\2617090873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['1_month_future_predict'] = result_df.groupby('youtube_user_id')['predict'].shift(-30).rolling(window=30).mean()
C:\Users\kkims\AppData\Local\Temp\ipykernel_23628\2617090873.py:8: SettingWithCopyWarning: 
A value is trying

In [ ]:
# 기간 따른 예측 결과
# 1개월
result_1_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] < result_df_final[result_df_final['predict'] > 0]['1_month_future_predict']]['youtube_user_id']
result_1_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] > result_df_final[result_df_final['predict'] > 0]['1_month_future_predict']]['youtube_user_id']

# 3개월
result_3_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] < result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']]['youtube_user_id']
result_3_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] > result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']]['youtube_user_id']

# 6개월
result_6_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] < result_df_final[result_df_final['predict'] > 0]['6_month_future_predict']]['youtube_user_id']
result_6_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] > result_df_final[result_df_final['predict'] > 0]['6_month_future_predict']]['youtube_user_id']

# 12개월
result_12_month_plus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] < result_df_final[result_df_final['predict'] > 0]['12_month_future_predict']]['youtube_user_id']
result_12_month_minus_user_id = result_df_final[result_df_final['predict'] > 0][result_df_final[result_df_final['predict'] > 0]['subscribers_count'] > result_df_final[result_df_final['predict'] > 0]['12_month_future_predict']]['youtube_user_id']

In [ ]:
print(len(result_df[result_df['youtube_user_id'].isin(result_12_month_minus_user_id)]['channel_title'].unique()))
print(result_df[result_df['youtube_user_id'].isin(result_12_month_minus_user_id)]['channel_title'].unique())

7
['유익한 균튜버' '0' '모하지연 MOHAJIYEON' '-mentalholder 멘탈홀더 tv' '지미 geemi.'
 '프롬수지 fromsuzy' 'fromsuzy 프롬수지']


In [ ]:
# 계속 증가
increase_user_id = result_df_final[result_df_final['predict'] > 0][(result_df_final[result_df_final['predict'] > 0]['1_month_future_predict'] < result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']) & 
                                                                   (result_df_final[result_df_final['predict'] > 0]['3_month_future_predict'] < result_df_final[result_df_final['predict'] > 0]['6_month_future_predict']) & 
                                                                   (result_df_final[result_df_final['predict'] > 0]['6_month_future_predict'] < result_df_final[result_df_final['predict'] > 0]['12_month_future_predict'])]['youtube_user_id']
result_df[result_df['youtube_user_id'].isin(increase_user_id)]['channel_title'].unique()

array(['낭만아저씨코디TV', '0', '오디디 코미디', '모하지연 MOHAJIYEON', '콜드쉽 Coldsheep',
       '팀브라더스', '맛집남자 foodman', '-mentalholder 멘탈홀더 tv', '황헬린 탈출기',
       '프롬수지 fromsuzy', 'fromsuzy 프롬수지'], dtype=object)

In [ ]:
# 계속 감소
decrease_user_id = result_df_final[result_df_final['predict'] > 0][(result_df_final[result_df_final['predict'] > 0]['1_month_future_predict'] > result_df_final[result_df_final['predict'] > 0]['3_month_future_predict']) & 
                                                                   (result_df_final[result_df_final['predict'] > 0]['3_month_future_predict'] > result_df_final[result_df_final['predict'] > 0]['6_month_future_predict']) & 
                                                                   (result_df_final[result_df_final['predict'] > 0]['6_month_future_predict'] > result_df_final[result_df_final['predict'] > 0]['12_month_future_predict'])]['youtube_user_id']
result_df[result_df['youtube_user_id'].isin(decrease_user_id)]['channel_title'].unique()

array([], dtype=object)